In [1]:
import pandas_gbq as gbq


In [4]:
cd training-data-analyst/blogs/gcp_forecasting

/Users/dangngochuy/training-data-analyst/blogs/gcp_forecasting


In [5]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
import time_series


In [7]:
#import libraries

import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import os


In [18]:
#set working directory 
path = "/Users/dangngochuy/Desktop/Hertie/Hertie 3rd semester/Machine Learning/Predicting-German-Election/Code/Transformed Data"
os.chdir(path)

#load data
#load data
afd = pd.read_csv("combined_data_afd.csv", index_col='date', parse_dates=True)
cdu = pd.read_csv("combined_data_cdu.csv", index_col='date', parse_dates=True)
fdp = pd.read_csv("combined_data_fdp.csv", index_col='date', parse_dates=True)
spd = pd.read_csv("combined_data_spd.csv", index_col='date', parse_dates=True)
lin = pd.read_csv("combined_data_lin.csv", index_col='date', parse_dates=True)
gru = pd.read_csv("combined_data_gru.csv", index_col='date', parse_dates=True)
oth = pd.read_csv("combined_data_oth.csv", index_col='date', parse_dates=True)

In [19]:
afd.head()

support
date                
2013-10-23  5.045125
2013-10-24  5.051701
2013-10-25  5.058277
2013-10-26  5.049102
2013-10-27  5.039927

## Feature Engineering for Machine Learning Model

In [69]:
series = pd.Series(afd['support'], index=afd.index)

In [99]:
WINDOW_SIZE = 30 
HORIZON = 3 # Predict approximately 3 days into the future
LABELS_SIZE = 1  # The label will be the voteshare of 1 day
df = time_series.create_rolling_features_label(series, window_size=WINDOW_SIZE, pred_offset=HORIZON)

/Users/dangngochuy/training-data-analyst/blogs/gcp_forecasting/time_series.py:98: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  .apply(_keep, args=(windows,))


In [100]:
df.tail()

-32_steps  -31_steps  -30_steps  -29_steps  -28_steps  -27_steps  \
pred_date                                                                      
2017-09-17   8.102721   8.138466   8.174212   8.209957   8.286518   8.363080   
2017-09-18   8.138466   8.174212   8.209957   8.286518   8.363080   8.439641   
2017-09-19   8.174212   8.209957   8.286518   8.363080   8.439641   8.525727   
2017-09-20   8.209957   8.286518   8.363080   8.439641   8.525727   8.591404   
2017-09-21   8.286518   8.363080   8.439641   8.525727   8.591404   8.658442   

            -26_steps  -25_steps  -24_steps  -23_steps  ...  -11_steps  \
pred_date                                               ...              
2017-09-17   8.439641   8.525727   8.591404   8.658442  ...   9.126160   
2017-09-18   8.525727   8.591404   8.658442   8.694867  ...   9.205651   
2017-09-19   8.591404   8.658442   8.694867   8.710884  ...   9.319466   
2017-09-20   8.658442   8.694867   8.710884   8.726902  ...   9.433281   
2017-09-21   8.694867   8.710884   8.726902   8.742919  ...   9.567504   

            -10_steps  -9_steps  -8_steps   -7_steps   -6_steps   -5_steps  \
pred_date                                                                    
2017-09-17   9.205651  9.319466  9.433281   9.567504   9.701727   9.825746   
2017-09-18   9.319466  9.433281  9.567504   9.701727   9.825746   9.949765   
2017-09-19   9.433281  9.567504  9.701727   9.825746   9.949765  10.073783   
2017-09-20   9.567504  9.701727  9.825746   9.949765  10.073783  10.177394   
2017-09-21   9.701727  9.825746  9.949765  10.073783  10.177394  10.284406   

             -4_steps   -3_steps      label  
pred_date                                    
2017-09-17   9.949765  10.073783  10.350602  
2017-09-18  10.073783  10.177394  10.416797  
2017-09-19  10.177394  10.284406  10.482993  
2017-09-20  10.284406  10.350602  10.527211  
2017-09-21  10.350602  10.416797  10.571429  

[5 rows x 31 columns]

In [101]:
# Features, label.
X = df.drop('label', axis=1)
y = df['label']

# Train/test split. Splitting on time.
train_ix = time_series.is_between_dates(y.index,
                                  end='2017-06-30')
test_ix = time_series.is_between_dates(y.index,
                                 start='2017-06-30',
                                 end='2017-09-21')
X_train, y_train = X.iloc[train_ix], y.iloc[train_ix]
X_test, y_test = X.iloc[test_ix], y.iloc[test_ix]

## Linear Regression

In [102]:
from sklearn.ensemble import RandomForestRegressor
# Train model.
cl = RandomForestRegressor(n_estimators=500, max_features='sqrt',
                              random_state=10, criterion='mse')

cl.fit(X_train, y_train)
pred = cl.predict(X_test)
random_forest_metrics = time_series.Metrics(y_test, pred)
random_forest_metrics.report("Forest Model")

Forest Model results
~~~~~~~~~~~~~~~~~~~~
RMSE: 0.292
MAE: 0.247
MALR: 0.026


In [103]:
test = y_test.tail(30).to_frame()

In [104]:
test.tail()

label
pred_date            
2017-09-17  10.350602
2017-09-18  10.416797
2017-09-19  10.482993
2017-09-20  10.527211
2017-09-21  10.571429

In [105]:
test_transposed = test.T
test_transposed.tail()

pred_date  2017-08-23  2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
label        8.525727    8.591404    8.658442    8.694867    8.710884   

pred_date  2017-08-28  2017-08-29  2017-08-30  2017-08-31  2017-09-01  ...  \
label        8.726902    8.742919    8.770841    8.798763    8.806277  ...   

pred_date  2017-09-12  2017-09-13  2017-09-14  2017-09-15  2017-09-16  \
label        9.825746    9.949765   10.073783   10.177394   10.284406   

pred_date  2017-09-17  2017-09-18  2017-09-19  2017-09-20  2017-09-21  
label       10.350602   10.416797   10.482993   10.527211   10.571429  

[1 rows x 30 columns]

In [106]:
election = cl.predict(test_transposed)
print(election)

[10.73779073]


In [107]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

lin_afd = LinearRegression()
lin_afd.fit(X_train, y_train)
lin_pred = lin_afd.predict(X_test)
linear_regression_metrics = time_series.Metrics(y_test, lin_pred)
linear_regression_metrics.report("Linear Regression")

Linear Regression results
~~~~~~~~~~~~~~~~~~~~~~~~~
RMSE: 0.059
MAE: 0.047
MALR: 0.005


In [108]:
test_lin = y_test.tail(30).to_frame()
test_lin_transposed = test.T
election_lin = lin_afd.predict(test_lin_transposed)
print(election_lin)

[10.6843783]


In [110]:
# Gradient Boosting Model 
from sklearn.ensemble import GradientBoostingRegressor

# Create GB model
gbr_afd = GradientBoostingRegressor()
gbr_afd.fit(X_train, y_train)
gbr_pred = gbr_afd.predict(X_test)
gradient_boosting_metrics = time_series.Metrics(y_test, gbr_pred)
gradient_boosting_metrics.report("Gradient Boosting")

Gradient Boosting results
~~~~~~~~~~~~~~~~~~~~~~~~~
RMSE: 0.163
MAE: 0.124
MALR: 0.013


In [112]:
test_gbr = y_test.tail(30).to_frame()
test_gbr_transposed = test.T
election_gbr = gbr_afd.predict(test_gbr_transposed)
print(election_gbr)

[10.75339729]
